In [1]:
library(mxnet)

In [2]:
# Import util functions
source("./common/utils.R")

# Import hyper-parameters
params <- load_params("cnn")

Loading required package: rjson


In [3]:
print(paste0("OS: ", Sys.info()["sysname"]))
print(R.version$version.string)
print(paste0("MXnet: ", packageVersion("mxnet")))

[1] "OS: Linux"
[1] "R version 3.4.3 (2017-11-30)"
[1] "MXnet: 1.2.0"


In [4]:
create_symbol <- function(){
  
  data <- mx.symbol.Variable('data')
  # size = [(old-size - kernel + 2*padding)/stride]+1
  # if kernel = 3, pad with 1 either side
  conv1 <- mx.symbol.Convolution(data=data, num_filter=50, pad=c(1,1), kernel=c(3,3))
  relu1 <- mx.symbol.Activation(data=conv1, act_type="relu")
  conv2 <- mx.symbol.Convolution(data=relu1, num_filter=50, pad=c(1,1), kernel=c(3,3))
  pool1 <- mx.symbol.Pooling(data=conv2, pool_type="max", kernel=c(2,2), stride=c(2,2))
  relu2 <- mx.symbol.Activation(data=pool1, act_type="relu")
  drop1 <- mx.symbol.Dropout(data=relu2, p=0.25)
  
  conv3 <- mx.symbol.Convolution(data=drop1, num_filter=100, pad=c(1,1), kernel=c(3,3))
  relu3 <- mx.symbol.Activation(data=conv3, act_type="relu")
  conv4 <- mx.symbol.Convolution(data=relu3, num_filter=100, pad=c(1,1), kernel=c(3,3))
  pool2 <- mx.symbol.Pooling(data=conv4, pool_type="max", kernel=c(2,2), stride=c(2,2))
  relu4 <- mx.symbol.Activation(data=pool2, act_type="relu")
  drop2 <- mx.symbol.Dropout(data=relu4, p=0.25)
  
  flat1 <- mx.symbol.Flatten(data=drop2)
  fc1 <- mx.symbol.FullyConnected(data=flat1, num_hidden=512)
  relu7 <- mx.symbol.Activation(data=fc1, act_type="relu")
  drop4 <- mx.symbol.Dropout(data=relu7, p=0.5)
  fc2 <- mx.symbol.FullyConnected(data=drop4, num_hidden=params$N_CLASSES) 
  
  input_y <- mx.symbol.Variable('softmax_label')  
  mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
  
}

In [5]:
cifar <- maybe_download_cifar()
x_train <- cifar$x_train
y_train <- cifar$y_train
x_test <- cifar$x_test
y_test <- cifar$y_test

rm(cifar)

In [6]:
cat('x_train shape:', dim(x_train), '\n')
cat('x_test shape:', dim(x_test), '\n')
cat('y_train shape:', length(y_train), '\n')
cat('y_test shape:', length(y_test), '\n')

x_train shape: 32 32 3 50000 
x_test shape: 32 32 3 10000 
y_train shape: 50000 
y_test shape: 10000 


In [7]:
sym <- create_symbol()

In [8]:
if (params$GPU) {
  ctx = mx.gpu(0)
} else {
  ctx = mx.cpu()
}

In [ ]:
# TODO: change BATCHSIZE back to default
params$BATCHSIZE <- 16

In [9]:
train_iter <- mx.io.arrayiter(x_train, y_train, batch.size = params$BATCHSIZE, shuffle = TRUE)

In [10]:
start.time <- Sys.time()

model <- mx.model.FeedForward.create(
  symbol = sym,
  X = train_iter,
  ctx = ctx,
  num.round = params$EPOCHS,
  learning.rate = params$LR,
  momentum = params$MOMENTUM,
  eval.metric = mx.metric.accuracy,
  initializer = mx.init.Xavier(rnd_type = 'uniform'),
  epoch.end.callback = mx.callback.log.train.metric(100)
)

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

Start training with 1 devices
[1] Train-accuracy=0.359254961587708
[2] Train-accuracy=0.50724
[3] Train-accuracy=0.56146
[4] Train-accuracy=0.59654
[5] Train-accuracy=0.61224
[6] Train-accuracy=0.62256
[7] Train-accuracy=0.6273
[8] Train-accuracy=0.62732
[9] Train-accuracy=0.62336
[10] Train-accuracy=0.62566


Time difference of 2.985388 mins

In [13]:
start.time <- Sys.time()

y_guess <- predict(model, mx.io.arrayiter(x_test, y_test, batch.size = params$BATCHSIZE, shuffle = FALSE))
y_guess <- apply(y_guess, 2, function(x) which(x==max(x))-1)

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

print(paste("Accuracy:", sum(y_guess == y_test)/length(y_guess)))

Time difference of 33.09693 secs

[1] "Accuracy: 0.6672"
